## Boilerplate

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from gensim.models import Word2Vec


## Word attention model with bias

In [141]:
import torch.nn as nn
import torch.nn.functional as F

class AttentionWordRNN(nn.Module):
    """
    The embedding layer + CNN model that will be used to perform sentiment analysis.
    """

    def __init__(self,  
                 num_filters=128, kernel_sizes=[3, 4, 5], freeze_embeddings=True, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(AttentionWordRNN, self).__init__()

        self.num_filters = num_filters
        #self.embedding_dim = embedding_dim

        KK=[]
        for K in kernel_sizes:
            KK.append( K + 1 if K % 2 == 0 else K)
            
        self.convs_1d = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embedding_dim), padding=(k//2,0)) 
            for k in KK])
        

        self.lstm = nn.LSTM(num_filters,
                           num_filters,num_layer,batch_first = True,bidirectional= True)
        
        self.dropout = nn.Dropout(drop_prob)
        self.sig = nn.Sigmoid()
    
    def conv_and_pool(self, x, conv,x1):

        x = conv(x)
        x = F.relu(x)
        x = x.squeeze(3)
     
        return x

    def forward(self, x):
        #print("#x##:", x.shape)
        x = torch.transpose(x,1,0)
        embeds = x.unsqueeze(1)
        conv_results = [self.conv_and_pool(embeds, conv,x) for conv in self.convs_1d]
        x = torch.cat(conv_results, 0)
        x = torch.transpose(x, 1,2)
        output,(final_hidden_state, final_cell_state) =self.lstm(x)
        x = torch.transpose(output, 0,1)
        x = torch.transpose(x, 1,2)
        x = F.max_pool1d(x, x.size(2))
        x = x.squeeze(2)
    

        return x.unsqueeze(0)
      
      
                

In [142]:
import torch.nn as nn
import torch.nn.functional as F

class AttentionSentRNN(nn.Module):
    """
    The embedding layer + CNN model that will be used to perform sentiment analysis.
    """

    def __init__(self, 
                 num_filters=128, kernel_sizes=[3, 4, 5], freeze_embeddings=True, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(AttentionSentRNN, self).__init__()

        self.num_filters = num_filters
        #self.embedding_dim = embedding_dim


        KK=[]
        for K in kernel_sizes:
            KK.append( K + 1 if K % 2 == 0 else K)
            
        self.convs_1d = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, 2*num_filters), padding=(k//2,0)) 
            for k in KK])
        

        self.fc1= nn.Linear(384, 1000) 
        self.lstm = nn.LSTM(num_filters, num_filters,num_layer,batch_first = True,bidirectional= True)
        self.dropout = nn.Dropout(drop_prob)
        self.sig = nn.Sigmoid()
  
    
    def conv_and_pool(self, x, conv,x1):

        x = conv(x)
        x = F.relu(x)
        x = x.squeeze(3)
        return x
    
    def forward(self, x):
        embeds = x.unsqueeze(1)
        
        
        conv_results = [self.conv_and_pool(embeds, conv,x) for conv in self.convs_1d]
        x = torch.cat(conv_results, 0)
        x = torch.transpose(x, 1,2)
        output,(final_hidden_state, final_cell_state) =self.lstm(x)
        x = torch.transpose(output, 0,1)
        x = torch.transpose(x, 1,2)
        x = F.max_pool1d(x, x.size(2))

        return x.squeeze(2)
                

## Sentence Attention model with bias

In [87]:
class entireContext(nn.Module):
    
    
    def __init__(self):                
        
        super(entireContext, self).__init__()

        self.wordCLSTM = AttentionWordRNN(
                       num_filters, kernel_sizes)
        self.senCLSTM = AttentionSentRNN(
                       num_filters, kernel_sizes)
        #self.CLSTM = CLSTM(output_size, 
        #               num_filters, kernel_sizes)
        self.Lin1 = nn.Linear(200,2)
 
    def forward(self, embed, source,max_sents):

    
        s = None
        for i in range(max_sents):
            _s = self.wordCLSTM(embed[i,:,:])
            if(s is None):
                s = _s
            else:
                s = torch.cat((s,_s),0)    

        y_pred1 = self.senCLSTM(s)
        #y_pred3 = self.CLSTM(source)
        #final = torch.cat([y_pred1,y_pred3], -1)
         
        y_pred = self.Lin1(y_pred1)
 
        return y_pred
    
   

## Functions to train the model

In [88]:
max_grad_norm=5

def train_data(mini_batch,source,  targets, sent_attn,  sent_optimizer, criterion):
    
    max_sents, batch_size, max_tokens, embed_size = mini_batch.size()
    sent_optimizer.zero_grad()
    y_pred = sent_attn(mini_batch,source,  max_sents)

    loss = criterion(y_pred.squeeze(), targets)
    loss.backward()

    torch.nn.utils.clip_grad_norm_(sent_attn.parameters(),max_grad_norm)
    sent_optimizer.step()
    
    return loss

In [7]:

def get_predictions(val_tokens1, val_tokens2,  sent_attn):
    max_sents, batch_size, max_tokens, embed_size = val_tokens1.size()
    y_pred = sent_attn(val_tokens1.cuda(),val_tokens2.cuda(),  max_sents)
    return y_pred

In [8]:
from gensim import models
embed_lookup = models.fasttext.load_facebook_model("kor_model.bin")

In [152]:
import gensim

embed_lookup = gensim.models.KeyedVectors.load_word2vec_format("wiki.ko.vec")

In [151]:
from gensim import models
embed_lookup = models.fasttext.load_facebook_model("cc.ko.300.vec", encoding='utf8')

NotImplementedError: Supervised fastText models are not supported

In [9]:
print(len(embed_lookup.wv.index2word))

80797


In [153]:
word2index={}
for i in range(0, len(embed_lookup.wv.index2word)):
    try:
        word2index[embed_lookup.wv.index2word[i]] = i
    except KeyError:
        word2index[embed_lookup.wv.index2word[i]] = i

/home/dlwngud3028/venv-python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/home/dlwngud3028/venv-python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [154]:
print(len(word2index))

879129


## Loading the data

In [155]:
# convert reviews to tokens
def tokenize_all_reviews(reviews_split):
    reviews_words = reviews_split.split(' ')
    tokenized_reviews = []
    for review in reviews_words:
        ints = []
        for word in review.split(' '):
            if(word==''):
                continue
            if(word=='.'):
                continue
            try:
                idx = embed_lookup.vocab[word].index
            except: 
                idx = 0
            tokenized_reviews.append(idx)
    return tokenized_reviews



In [156]:
import csv
fr = open('trainset.tsv', 'r', encoding='utf-8')
rdrr = csv.reader(fr,  delimiter='\t')
it =0
for line in rdrr:
    it+=1
   # print(sen)
    
print(it)
fr.close()


fr = open('testset.tsv', 'r', encoding='utf-8')
rdrr = csv.reader(fr,  delimiter='\t')
it =0
for line in rdrr:
    it+=1
    
print(it)
fr.close()


20843
3176


In [157]:
import random
result=[]
fr = open('trainset.tsv', 'r', encoding='utf-8')
rdrr = csv.reader(fr,  delimiter='\t')
it =0
X_train=list()
y_train=list()
for line in rdrr:
    it+=1
    #print(line)
    a = line[0]
    b = line[1]

    if '1' in b:
        b=int(b)
        b = 0
    elif '2' in b:
        continue
        b=int(b)
        b = 1
    elif '3' in b:
        b=int(b)
        b = 1
    else:
        continue
        
    if(a==''):
        continue
    #print(a)
  
        
    a_t=' '
    j=[]


    for e in a.split(' '):
        a_t +=e+' '
        
        c_t = list(tokenize_all_reviews( a_t))
        
        if(len(c_t) <=3):
            continue
            
        if('.' in e):
            b_t = list(tokenize_all_reviews(a_t))
            j.append(b_t)
            a_t=' '
        elif('?' in e):
            b_t = list(tokenize_all_reviews(a_t))
            j.append(b_t)
            a_t=' '
        elif('!' in e):
            b_t = list(tokenize_all_reviews(a_t))
            j.append(b_t)
            a_t=' '

        
    if(len(j) == 0 ):
        b_t = list(tokenize_all_reviews(a))
        j.append(b_t)
    elif(len(j) > 0 and len(a_t)>=3):
        b_t = list(tokenize_all_reviews(a_t))
        j.append(b_t)


    if(len(j)==0):
        continue


   #print("############################################")
    X_train.append(j)
    y_train.append(b)

    

fr.close()

print(len(y_train))


12329


In [158]:
import random
result=[]
fr = open('testset.tsv', 'r', encoding='utf-8')
rdrr = csv.reader(fr,  delimiter='\t')

it =0
X_test=list()
y_test=list()
data_test=[]
for line in rdrr:
    it+=1
    #print(line)
    data_test.append(input_sentence)
    a = line[0]
    b = line[1]

    
    if '1' in b:
        b=int(b)
        b = 0
    elif '2' in b:
        continue
        b=int(b)
        b = 1
    elif '3' in b:
        b=int(b)
        b = 1
    else:
        continue
        
    if(a==''):
        continue
    print(a)
  
        
    a_t=' '
    j=[]


    c_t=' '
    for e in a.split(' '):
        a_t +=e+' '

        c_t = list(tokenize_all_reviews(a_t))
        
        if(len(c_t) <=3):
            continue
        if('.' in e):
            b_t = list(tokenize_all_reviews(a_t))
            j.append(b_t)
            a_t=' '
        elif('?' in e):
            b_t = list(tokenize_all_reviews(a_t))
            j.append(b_t)
            a_t=' '
        elif('!' in e):
            b_t = list(tokenize_all_reviews(a_t))
            j.append(b_t)
            a_t=' '
        

    if(len(j) == 0 ):
        b_t = list(tokenize_all_reviews(a))
        j.append(b_t)
    elif(len(j) > 0 and len(a_t)>=3):
        b_t = list(tokenize_all_reviews(a_t))
        j.append(b_t)
        
    if(len(j)==0):
        print("EE")
        continue

   # print(j)
   #print("############################################")
    X_test.append(j)
    y_test.append(b)

    
read_f.close()


 !!!! 남포동, 서면, 시청, 부산대에 가시면 우리 시민들 피 빨아먹는 거머리 같은 풍산의 특혜 비리 및 부산시가 무엇 하는 곳인지 잘 알 수가 있어요!! 꾹 들러 부산시민의 힘을 보여주세요
 ＂文 정부, 김기식 무너지면 참여연대 이어 청와대 무너질까 두렵나＂ 검찰 조사에 이어 관철할 것! 김기식 밑바닥 드러나면, 참여연대의 위선적 밑바닥도 드러나니 두려워하는 것. 
 (contd 교육에서는 a 유형에게는 칭찬만 하는 편이 좋고, b 유형은 칭찬만 들으면 싱겁다고 느끼거나 위선, 가식, 무관심은 아닌지 의심하기 때문에 있는 그대로 말하는 편이 좋다고 한다.
 (미노스는 오지랖을 물리쳤다. 
 (쌤통이다.
 .ㅌ 이게모람! 태양열받아 춤추는 인형인가!(상상하는 꼬락서니
 . 탐라에 합성 가능하신 분 계시나요 로고나 보석들.(개빡침 합성 따위 못하는 인간이라 그리고 있는데 사람 할 짓이 못 되는 것 같습니다 커미션이 있다면 추천 좀 해주세요.
 .팬레ㅌ 솔직히 병 크러들 키우는데 기획사가 한몫했다는 생각 든다. 플미충떠도 프로미 안 잡음. 잡는 거 힘들지 그럼 공지라도 한 번 해야지. 티켓은 책임질 수 없습니다.이런 식으로 위협이라도 줘야지ㅠ
 . 괴담. 오토디셉 군대 괴담 넘쳐나지만 그 누구도 과학부의 미스터리와 의료부의 알아선 안 되는 진실들<을 깬 괴담들을 없었다
- .저기요. - .네? 윙의 목소리에 딥 고개 갸우뚱하다가 윙 아무것도 아니라며 손목 잡은 손 놓고는 일어나서 어디론가 가려고 하겠지 근데 이 남자, 잘 곳이 없는 건가. 그 생각이 들자마자 무슨 오지랖인지는 모르겠는데 딥 윙 손목 잡아버린다
 : (6 fta 전체가 국내법보다 우위에 있다? 생쇼를 해라. 세계화에 따라 국내법체계가 국제적 관행을 점진적으로 반영하게 되는 것은 당연하다. 이것은 케이스 바이 케이스다
 : 원맥소오 헤어스타일 반전(머리 내리고 단발로 자르고!입니.는 개뿔 자컾 만들 엇냐 김라 언니 : 너 내 거! 제일 좋아해! 사랑해!! : 웅 그래 (개에 크 
 : 최시중 환자

[  event ] 이번 마스터가 출국 전 처음이자 마지막 오프라 너무 간절한데 플미충들 꼴보기 싫어서 티켓 구할 때 수고비로 붙이려고 모아둔 돈으로 >>갈릴레오 대형<< 망원경 알티 추첨 한 분 사드려요 만약 원가 양도(기왕이면 첫날 받게 되면 망원경 + 제 것 사면서 이 밤 2도 같이 보내드립니다 
[ 개인적으로 과거사를 이야기하는 것을 좋아하지 않아. 이미 역겹고 수치스러운 기억이라고 정해버렸기 때문에.] 
[ 극과 극의 추천인거욬 조용하고 개인주의적인 애쉬랑 활발을 넘어 과도하고 오지랖 넓은 빈터랑 페어플레이햌 ???? ]
[ 있지 바쿠고. ] [ 왜. ] [ 지금 기분 좋아? ] [ 뭔 짓 했냐. ] [ 미안해! 반지 잃어버렸어. ] [ ㅉㅉ이 칠칠맞은 새끼. ] [ 화 안 났어? ] [ ㅊ 새로 사러 가지 뭐. ] [ .바쿠 고도 사실 잃어버렸지. ] [ 많이 티 났냐? ]
[ 필리버스터 ] 김경협 의원 발언 중 조원진 의원 항의 이석현 조원진에게 개빡침 <경고했어요, 퇴장시키기 전에 들어가세요> <내가 의장직을 걸고 얘기합니다. 의사진행 방해하지 마세요>
[!트친소!] 이런 그림 그리는 새럼입니다 마음 남겨주시면 취향 맞는 분 찾아가요.! 3라인은 위주 리버스도 좋아하는 원더러 낼 루미입니다 탐라대 화랑 섹트, 빻은 취향 등등 유 위해서 흔적 남겨주세요.! 
[] 180309(금 안내 판매처 : 한남 동북 파크 (서울시 용산구 한남동 727-56, 블루스퀘어 3층 사인회 시간/장소 : pm 8:00 (tcc 아트센터 / 서울특별시 영등포구 당산동 4가 93-1 
[★] 가식과 위선. a puppet with a pretending 
[10월의 퀴어 밥상] 명절날 고된 집안 일과 친척들 오지랖에 지치신 분들은 퀴어 밥상으로 오세요.영양 듬뿍 버섯밥과 주키니 팬케이크와 함께 수다 수다하면 몸도 마음도 힐링! 10월 7일 오후 1시까지 kscrc로 오시안 
[13화 예고] , ＂나한테 시집올래?＂ 향한 수작질 
[13화 예고] 나한테 시집올래? 오공이의

[이슈플러스] 靑·국방부, 사드 환경영향평가 이견. 새 쟁점 부상 | 다음 뉴스 나라가 오염되고 망쳐진 후에 안 보는 무슨 개뿔? 안보만큼 소중한 게 환경이라고. 그런데 소규모 환경평가만으로도 만족하겠다고?
[이재정 의원실] 2014년 이후 182억 원의 공무원연금이 과 오지급 되었음에도 불구하고 그 63%를 회수하는 데 그쳤습니다. 공무원연금이 국민의 세금으로 일부 보전되고 있는 만큼, 운영의 내실화에 힘써야 합니다. 
[이종택 칼럼] 김진태 지지 결집을 부추기는 인명진의 얄팍한 수작! 2선에 불과한 김진태의 대선 후보 부상은 반란 세력 그중에서도 비박으로 분류되는 역적들에게는 생각지도 못한 변수다 공포다. 그렇지. 
[이중 위선자! 단 하루도 못 참겠다, 박근혜는 즉시 내려와라!][단독] 특검 수사 받겠다던 박 대통령, 측근에겐 특검 막아라 지시 | 다음 뉴스 
[인터뷰] 뉴스공장 김어준, 촌스럽고 위선적인 시대를 사는 국민에게 위로가 되길 
[인터뷰] 박지원 탄핵 통과가 목표. 비박 계속 설득 비박은 이미 돌아선 상태다 그래서 추미애가 무성히 만난 거란다 무성히 가 말 바꾸기 도사였던 거 몰랐나 목적은 개헌을 통해서 나눠 먹는 거였지 핑계는. 오지랖
[인터뷰] 선우정아 - 오지랖 예술의 혼을 불태우는 그녀, 음악적 원천은 열등감 장담하건대 이번 인터뷰는 선우정아라는 뮤지션과 그녀의 음악에 한 발 더 가까이 다가가는데 좋은 매개체가 될 것이다.
[인터뷰] 이병태 kaist 교수 정부의 오락가락 행보. 가상화폐 가격 출렁 ① 이 교수 대단하다. 비트코인에 세금을 부과하면 가상화폐 기술을 원천 차단하는 비판을 받을 수 있대. 도대체 카이스트 교수들 여기에 얼마나 투자한 거냐.
[일상] 스킨스쿠버 배우기_일산 수작 / 출처 : 륜이 여사_ li. | 블로그 
[전문] 문재인, 병역 면탈·논문 표절 등 5대 비리 관련자 고위공직 원천 배제 병역 면탈, 부동산 투기, 세금 탈루, 위장전입, 논문 표절 등 5대 비리 관련자는 고위공직에서 원천적으로 배제해야 합니다. 

48억 세금 낭비 막은 직원. 일자리 잃고 홀로 법정 싸움 | 다음 뉴스 
4구 대회가 없어진 이유 내가 하면 기분 좋은데 당하면 개빡침 ㅇㅈ? 
4년 전, 지푸라기라도 잡는 심정으로. 그곳만은 가지 말아야지 했던. 위선의 대명사(그때의 편견 교회라는 곳을 처음 가게 됐을 때. 4년 뒤 제가 기독교 대표 방송의 강의 프로에 나가게 될 줄은 꿈에도 몰랐죠. 이런 제가 어떻게 하나님을 부정하겠습니까.
4부 미국 싱크탱크 지원 중단과 김기식-홍일표 라인의 놀라운 이중성과 위선 [사회 이슈] (2018.04.09 님이 공유
4월에 눈이 오는 꼬락서니를 보면. 8월에도 겨울이 오는 날이 머지않은 거 같아 주나.
4조 원이 넘는 불법 차명계좌를 세금 한 푼 안 내고 이건희에게 돌려준 금융위는 삼성의 심부름센터인가요?. 전면 재조사해서 최소 2조 원 이상 세금 확정하고 즉각 추징하라! 
4차 산업은 잘 모리겠는데, 5차 산업으로 종교 산업을 육성하면 어떨까 싶다만. 전 세계에서 우리나라 만치 종교 사업(?이 발달한 나라가 없는데. 벤처 종교 회사를 설립해서 투자도 받고 능력 있는 종교인은 연봉도 많이 받고, 또 세금도 많이 내고.
5,18 유공자는 가산점 10% 그러면 공부 열심히 해도 공무원 될 수 없다 모든 세금 전부 공짜다 이것이 공정한 사회가 되겠습니까 국가를 위해 희생된 분들도 이렇게 안됩니다 이제 국민 여러분 자녀들 아무리 고생해도 공무원 임용 어렵습니다
5,18특별 법과 세월호 특별법 모두 폐기해야 한다 국고 축내는 하마다 왜 세금으로 이들을 보호해야 하나 새 정부는 바로 폐기 바란다 
5. 사생 대처 안 함 6. 황제님한테 염분 알레르기가 있음에도 불구하고 바다 가서 촬영 7. 오히려 이니스프리가 일 더 잘함 8. 샤콘 티켓 플미충들 단속한다고 입으로만 함 행동으로 실천 안 함
5.18 유공자 명단 즉시 공개해야 한다 그 명단을 공개하고 안 하고는 보훈처나 국가기관이 권리가 있는 게 아니라 세금을 내는 국민들이 그 세금이 어떻게 수용되고 있는가를 철저히 제대

 개 웃겨 자기들이 한 건 일단 아니라고 하고 우기기 너희들도 아미 일부만 보고 몇 날 며칠 가수 닮아 빻았다고 염불 외면서 일부 래디컬 페미놈들 빻은 행동은 못 본 척하니ㅋㅋ너희들이 보기에도 엿 같은 발언이긴 하지?. 자긍 
 개빡침 
 개빡침 
 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침 개빡침
 개빡침 머했다고 님원고안했져!!!! 유희왕만ㄴ쳐봤지 슬아기야!!! 기ㅣㅣㅣ (훈훈
 개빡침 이준호 김준수 땜에 화남ㅋㅋㅋㅋ 보는사람까지덥게만들어섴 
 개빡침 ㅌ 
 개빡침.내가 폭탄을 엘리베이터 밖으로 던졌는데. 폭탄에 자석이 있어서. 다시 돌아왔어
 개빡침;ㅠ
 개뿔. 빅 엿 먹은 기분임
 거짓말도 작작해라. 무슨 백두혈통이노. 김일성 친조부까지 전라도 전주가 뿌리인데. 전주 모악산에 있는 김일성 조상들 묘가 너희가 전라도 전주가 뿌리인 전주김씨 혈통인 걸 입증하는데. 백두혈통은 개뿔.
 경ㅁ멸하다는 뜻의 snub나 contempt 같은 거 너무 역겹고. 경멸하다는 despise써야겟ㄷ다. 여혐낭낭단어라서 외우겠네요!!!!! 시 펄
 경희대 빻글 제보합니다 미투 운동을 자신의 연애 성공 수단으로 이용하려는 빻은 경희대 남 
 고맙긴 개뿔 사과나 해라 이 미친 것들아 언제까지 눈 막 귀 막 할 거야 에로 남불 오지는 씹에리들 너넨 진짜 처맞아야 함 
 공용 책상에 이런 게 적히고, 반 안에서 돌려쓰는 일기에 학교 앞에서 병아리나 팔 메가를 년이라는 둥 인신공격당함 게다가 쌍욕 섞인 뒷담도 깐다더라 
 관변 단체 국고보조금 싹 다 없애야 합니다. 돈 받고 오는 사람들이 대부분일 텐데. 우리 세금이 저런데 쓰인다 생각하니 피가 거꾸로 솟습니다! ㅆㅂ
 광고 모델 잘못 × 광고 모델 잘못 × 광고 모델 잘못 × 광고 모델 잘못 × 광고 모델 잘못× 모르면 외우세요. 지금 이게 모델이랑 뭔 관련이 있냐

국민의당文대통령, 시진핑에게 기자 폭행 유감 표명하라 와 호래자식들 이 순간에 어떻게든 대통령과 엮고 기자들한테 잘 보이려고 수작 부리네 징글징글하다 통합 꿈도 꾸지 마 새끼들아
국정원 세금 7억이나 삥땅쳐서 해 입은 옷들, 더럽게 촌스러워요. 순실이 취향하고는! 
국정원 언론장악 문건과 함께 더불어민주당 언론장악 문건도 조사했으면 합니다만 안 하면 그거야말로 정치검찰 아니겠습니까 사법부 독립은 개뿔 장악 오지고요 국정원 언론장악 문건 있다 방송 간부· pd도 블랙리스트 
국정원 특수활동비를 박근혜 청와대가 상납 받았답니다 그리고 조윤선. 연기 완도 매달 500만 원이나 상납 받았음 이것들 현직 근무할 때 자기들이 최고 우아하고 고상한척하더니 위선자들 이런 것들이 국민을 위한다고 국정원 돈이 국정원께 아니며 국민의 피 같은 돈인데 도적 놈들
국정원이 가짜 뉴스 공장이었다. 독일처럼 1건당 600억씩 벌금 물려라. 국민 세금으로 봉급 받는 자들이 국민을 속이는 이런 범죄를 저지르다니. 마땅히 호된 벌을 받아야 한다. 
국정원한테 돈(세금 받은 보수논객 80인 명단 공개. 국정원 개들 - 전원책, 변희재, 조갑제, 김진, kbs 사장 고대영 등 전원책. 잘난체는 젖나 게 하는 놈이 결국 돈 받고 짖어대는 개였어. 
국회 거머리 같은 종자들! 
국회 꼬락서니 돌아 보고 떠들어라. 전과자 성폭행 뇌물 온갖 불법비리 연루자 천지가 국회야 .전과자 들에 나라를 맡기지고?? 국회가 범죄자 피난소다. 
국회는 치외법권 지역이라 인식하는 혼수 성태의 갑질이 참으로 가관이다. 이런 새끼들 언제쯤 안 보고 살 수 있을까? 
국회의 원전 번이 개인 정보였어? 미친 국가들. 자유 당국 개들 선거 때 문자 보낸 것도 같은 방법으로 고발해야지. 국민을 상대로 입막음해보려는 수작일진데 끝장 보자 자유한국당, 국회의원에 대규모 비방 문자 검찰 고발 
국회의원은 법위에 군림하는 귀족입니까, 신의 아들입니까? 맞습니다 그들은 귀족이고 신의 아들입니다. 다 아시면서. 한때는 이랬습니다. 그때만 

나도 전ㅎㅁ 싫어하지만, 남의 연애사에 누가 못생겼니 누가 한남이니 말하는 것은 그의 애인에게 실례가 아닌가. 또 그와 별개로 그의 행실이 별로였음에 월평이 당연해질 수 있는가? 나는 아니라고 보는데. 비판의 수단으로 미러링을 써야지 미러링이 주 목적이 되어서는 안 된다고 생각해.
나도 한때는 광팬이었는데 요즘은 석희 저놈 이름도 듣기 싫다. 역겹다. 
나도 항상 성괴라는 표현 들을 때마다 짜증이 난다. 성형을 해서 예뻐진 연예인들에는 열광하면서 성형이 예쁘게 안된 사람들에게는 어찌나들 가혹한지.
나라 꼬락서니가 어찌 되려는지? 국민의 70.80% 탄핵에 찬성한 박근혜를 아직도 아직도 옹호하는 인간들이 있으니. 아녀자가 어디 나가서 남의 애를 배어와도 할 말은 있고 억울한 점이 있다 했다. 무식한 것들.
나라 꼬락서니가 왜 이 모양 이 꼴이냐? 테러범에 국민이 납치 되질 않나 백주에 인질사태가 나질 않나. 하루 걸러 사고 나니 이것 불안해서 살겠나. 썩을 놈들
나르샤 위선자들이 틈새다 당주도 권 뺏어 쥐려고 지도부 물러나라는 연판장 돌려. 그럼 즈그들은 늘 뭐 했나? 불과 반년 전까지 당 대표하던 김무성 이 책임을 모두 현지지도부에 돌리는 꼴이 가증스럽다.
나를 뮤즈로 두는 것에 거부감은 없지만, 내가 뮤즈라고 해놓고 결과물이 형편없으면 진심 개빡침 뭐라구? 그 토사물이 나란 말이야? 정말 죽여버리고 싶어!
나만 그런 거 아니었구나 나 아까 5시간 동안 무슨 생쇼 한 거지 현타 개세게 옴
나만 애니 얘기나 후원 나올 때마다 채팅창에 ㄴㄷㅆ 역겹나??? 애니도 연예인이나 크리에이터같이 똑같이 덕질하는 장르 중에 하나인데 그걸 자꾸 배척하는 느낌이 듦. 아이돌 후원 나오면 그렇게 우리 언니 오빠 누나 형 거리면서 애니한테만 그럼 나도 아이돌 파지만 그런 거 보면 참.
나미 친 년 바른 정당 남경필 대선 출마 축하식 최고 회의에 참석 그냥 나대지 말고 새누리당을 떠나라 거머리 같은 년 
나쁘지 않아요 진짜 울고 싶어요 흑흑 어차피 현판 뛰어도 못 구할 거,, 양도충

법이 빻은 건 맞지만 제재가 있고 나이 제한을 걸어 둔 성인물을 뚫는 법. 그걸 공공연하게 공유하는 건 아니라고 봄
법적 대응은 개뿔,, 반성을 모르는 미친 인간들,,  안철수 대표 용산소방서 방문, 소방관들 격분, “대통령 놀이 그만하라” 댓글 폭발, 국민의당 법적 대응 ← 어익후. 
베충 를 딱 들 이 제천 유가족 동영상 퍼트리면서 문지 지자들 자극함 악플 달기를 바라는 것 같은데 절대 넘어가면 안 됨.
베충이들 북한 못 잃어하는 거 보니 역시 일성ㄷㄷ북한얘기만나오면 충에 벌떡 뛰어오는 거 보소 네댓 자꾸 뒤집히는 거 보니까 개빡침.ㅇㅏ.
변명 덩어리 안철수!! 이런 무책임하고 위선적인 사람이 대선에 나갈 생각을 하는 거지? 파렴치한이다! 
변명이 많은 사람은 위선적인 사람이 많다 이유가 많은 사람은 거짓된 사람이 많다 핑계가 많은 사람은 가식적인 사람이 많다 과시욕이 많은 사람은 허세에 능한 사람이 많다 욕심이 많은 사람은 외골수적인 사람이 많다 
변승욱 대기자 우리 언론이 미국 세금 탈류 증세 걱정하나 미국 기업을 대변한다 울 대통령 트럼프 의중을 알고 있었고 우린 이케 할 것이다 비행기 안에서 자료까지 주면서 부탁함 미 폭스 기사는 쓰지도 않고 중요한 기사는 빼버린 자국 대통령 죽이는 언론 개시 키들 
변호사 출신 이재명이 성남시장 하는 동안 법 잘못 철거민과 힘없는 사람들 상대로 온갖 고소고발을 일삼더니. 자기부정이 끝내주죠. 이재명 토론하는 꼬라니 보니, 지난 청문회 때 거짓 들통나 엉덩이 들썩거리며 어쩔 줄 몰라 하던 김기춘이 떠올랐습니다. 인생 진짜 더럽게 사는 이재명. 분명 벌받을 겁니다.
변희재 대표 정치검찰한테 개빡침 ㄷㄷㄷ 민민 껌 찰 
별 멍멍이 새끼들은 선의면 아무 데나 다 오지랖 떨어도 된다고 생각해서 큰일이다.
별로 관심도 없었지만, 알게 되었더니 기분이 나쁘다. 하지만 한 편으로는 그저 화풀이, 증오하고 싶은 대상을 찾아 다 쏟아내는 것으로 밖에 보이지 않아서 그 당당한 행태가 역겹기도 하다. 그냥 다 같이 자멸하자고 말하는 것

In [159]:
print(len(y_train))
print(len(y_test))

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_train.shape)

12329
3175
(12329,)


In [160]:
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.metrics import roc_auc_score

In [161]:
from numpy  import array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_train.shape)
print(X_test.shape)

print(X_train[4])
print(X_test[15])

(12329,)
(3175,)
[[0, 0, 0, 0, 69890, 41803, 7245, 373, 0, 0]]
[[0, 62755, 68419, 321700, 0], [168, 446271, 1813, 0, 0], [3294, 0, 152467, 14013, 0, 0], [0]]


In [162]:
k=0
weights = list()

for i in range(0, len(embed_lookup.wv.vocab)):
    cc = embed_lookup.wv.index2word[i]
   # c = embed_lookup.vocab['shit'].index
   # print(c)
    try:
        #print(fm[cc])
        weights.append(np.ndarray.tolist(embed_lookup[cc]))
    except KeyError:
        weights.append(np.ndarray.tolist(np.random.rand(300,)))
    k+=1     
    
weights = np.array(weights, dtype=np.float32)
weights = torch.from_numpy(weights)

weights = torch.FloatTensor(weights)
print(len(weights))

/home/dlwngud3028/venv-python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/home/dlwngud3028/venv-python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


879129


In [163]:
from numpy  import array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_train.shape)
print(X_test.shape)

print(X_train[6])
print(X_test)

(12329,)
(3175,)
[[0, 18209, 94637, 459, 0, 15678, 0, 34894, 18209, 94637, 459, 4180, 0, 1601, 18209, 380403, 45, 165931, 35267, 728, 0], [0]]
[list([[0, 0, 0, 0, 673299, 146306, 885, 61047, 12420, 231569, 275751, 54, 0, 66061, 18193, 39, 278130, 55632, 89, 231090, 218, 489, 921, 0], [173942, 56926, 0, 1222, 171853]])
 list([[0, 0, 482311, 266862, 68012, 501, 7403, 0, 0, 10659, 9675, 501, 418526, 0], [482311, 87673, 0, 281785, 393506, 0, 0, 55516, 0]])
 list([[0, 234773, 47, 0, 0, 89, 5180, 0, 102, 41643, 0, 42830, 0, 169308, 0, 0, 233023, 14256, 123254, 70, 26, 670, 1729, 5180, 8284, 0]])
 ...
 list([[1263, 486, 1912, 0], [0, 590, 0, 19075, 0, 0, 0, 0, 479563, 506, 1428, 24789, 0], [0]])
 list([[32943, 169385, 12535, 0], [0]])
 list([[32943, 9370, 76, 17021, 49262, 560340, 0, 0, 184006, 27202, 0, 43779, 0, 54, 0, 0, 2820, 296978, 318625, 0, 1677, 51643, 0], [98673, 6829, 8128, 20132, 387, 113579, 8390, 22328, 0]])]


In [164]:
def pad_batch(mini_batch):
    mini_batch_size = len(mini_batch)
    max_sent_len = int(np.max([len(x) for x in mini_batch]))
    max_token_len = int(np.max([len(val) for sublist in mini_batch for val in sublist]))

    if(max_sent_len==1):
        max_sent_len=2
        
    if(max_sent_len >= 100):
        max_sent_len = 50
    if(max_token_len >= 500):
        max_token_len = 100
        
   # print("Max_token_len: ", max_token_len)
   # print("Max_sent_len: ", max_sent_len)
    main_matrix = np.zeros((mini_batch_size, max_sent_len, max_token_len), dtype= np.int)
    for i in range(main_matrix.shape[0]):
        for j in range(main_matrix.shape[1]):
            for k in range(main_matrix.shape[2]):
                try:
                    main_matrix[i,j,k] = mini_batch[i][j][k]
                except IndexError:
                    pass
    return Variable(torch.from_numpy(main_matrix).transpose(0,1))

In [165]:

def test_accuracy_mini_batch(tokens1, tokens2, labels,  sent_attn):
    y_pred = get_predictions(tokens1, tokens2, sent_attn)
    _, y_pred = torch.max(y_pred, 1)
    
    correct = np.ndarray.flatten(y_pred.data.cpu().numpy())
    labels = np.ndarray.flatten(labels.data.cpu().numpy())

    num_correct = sum(correct == labels)
    return float(num_correct) / len(correct)

In [166]:
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.metrics import roc_auc_score

def test_accuracy_full_batch(tokens,  labels, mini_batch_size,  sent_attn):

    sent_attn.eval()

    p = []
    p2 = []
    l = []
   
    probs=[]
    #print("SDfsfsdf " ,len(tokens))
    g = gen_minibatch2(tokens, labels, mini_batch_size)

    for tokens1, tokens2, label in g:
        embedding = nn.Embedding.from_pretrained(weights)
        tokens1 = embedding(tokens1.long())
        tokens2 = embedding(tokens2.long())
        
        y_pred3 = get_predictions(tokens1.cuda(),tokens2.cuda(),  sent_attn)
        _, y_pred1 = torch.max(y_pred3, 1)
        

        p.append(np.ndarray.flatten(y_pred1.data.cpu().numpy()))
        l.append(np.ndarray.flatten(label.data.cpu().numpy()))
        


    p = [item for sublist in p for item in sublist]
    l = [item for sublist in l for item in sublist]

    #print(p)
  
    p = np.array(p)
    l = np.array(l)

    recall=0
    precision=0
    ROC=0
    

    print("test 갯수 :", len(p))
    precision = precision_score(l, p, average='micro')
    recall = recall_score(l, p, average='micro')

    
    F1score = f1_score(l,p,average='micro')


    
    #ROC  = roc_auc_score(l, p2)
    print("F-1score : ", F1score)

    print("AUC : ",ROC )
    
    num_correct = sum(p == l)
    
    
    print(p)
    print("############")
    print(l)
    print("맞춘갯수 : ", num_correct)
    return F1score, (float(num_correct))/ (len(p)), precision, recall

In [167]:


def test_data(mini_batch, source, targets, sent_attn):    
    max_sents, batch_size, max_tokens, embed_size = mini_batch.size()
       
    y_pred = sent_attn(mini_batch,source, max_sents)
    

    loss = criterion(y_pred.squeeze().cuda(), targets) 
 
    
    return loss.data.cpu()

In [168]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [169]:
def gen_minibatch(tokens, labels, mini_batch_size, shuffle= True):
    for token, label in iterate_minibatches(tokens, labels, mini_batch_size, shuffle= shuffle):
        token1 = pad_batch2(token)
        yield token1, Variable(torch.from_numpy(label), requires_grad= False).cuda() 

In [170]:

def gen_minibatch2(tokens, labels, mini_batch_size, shuffle= True):
    for token, label in iterate_minibatches(tokens, labels, mini_batch_size, shuffle= shuffle):
        token1 = pad_batch(token)
        token_x=[]
        for i in token:
            c = []
            for j in i:
                for k in j:
                    c.append(k)
            token_x.append(c)

        token2 = pad_batch2(token_x)

        yield token1, token2, Variable(torch.from_numpy(label), requires_grad= False).cuda() 

In [171]:
def pad_batch2(mini_batch):
    mini_batch_size = len(mini_batch)
    max_sent_len = int(np.max([len(mini_batch[x]) for x in range(0,len(mini_batch))]))
    if(max_sent_len==1):
        max_sent_len=2
        
    if(max_sent_len >=300):
        max_sent_len = 100
        
    #print("max_sent_len:", max_sent_len)
    main_matrix = np.zeros((mini_batch_size, max_sent_len), dtype= np.int)
    for i in range(main_matrix.shape[0]):
         for k in range(main_matrix.shape[1]):
            try:
                main_matrix[i,k] = mini_batch[i][k]
            except IndexError:
                pass
    return Variable(torch.from_numpy(main_matrix))

In [172]:
def pad_batch(mini_batch):
    mini_batch_size = len(mini_batch)
    max_sent_len = int(np.max([len(x) for x in mini_batch]))
    max_token_len = int(np.max([len(val) for sublist in mini_batch for val in sublist]))

    if(max_sent_len==1):
        max_sent_len=2
        
    #print("max_sent_len:", max_sent_len)
    #print("max_token_len#:", max_token_len)
    if(max_sent_len >= 50):
        max_sent_len = 20
    if(max_token_len >= 300):
        max_token_len = 100
        
    main_matrix = np.zeros((mini_batch_size, max_sent_len, max_token_len), dtype= np.int)
    for i in range(main_matrix.shape[0]):
        for j in range(main_matrix.shape[1]):
            for k in range(main_matrix.shape[2]):
                try:
                    main_matrix[i,j,k] = mini_batch[i][j][k]
                except IndexError:
                    pass
    return Variable(torch.from_numpy(main_matrix).transpose(0,1))

In [173]:
  
def check_val_loss(val_tokens, val_labels, mini_batch_size, sent_attn):
    val_loss = []
    for token, label in iterate_minibatches(val_tokens, val_labels, mini_batch_size, shuffle= True):
        token_x=[]  
        for i in token:
            c = []
            for j in i:
                for k in j:
                    c.append(k)
            token_x.append(c)
            
        
        embedding = nn.Embedding.from_pretrained(weights)

        tokens1 = embedding(pad_batch(token).long())
        tokens2 = embedding(pad_batch2(token_x).long())
        loss =test_data(tokens1.cuda(),tokens2.cuda(),  Variable(torch.from_numpy(label), requires_grad= False).cuda(), 
                                    sent_attn)
        val_loss.append(loss)
    return np.mean(val_loss)

In [174]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [175]:

print(len(weights))

879129


## Training

In [180]:



def train_early_stopping(mini_batch_size, X_train, y_train,X_dev,y_dev,X_test, y_test,
                          sent_attn,  sent_optimizer, loss_criterion, num_epoch ):
    start = time.time()
    loss_full = []
    
    
    epoch_counter = 0
    sent_attn.train()
    

    cur=0
    for i in range(1, num_epoch + 1):
        loss_epoch = []
        g = gen_minibatch2(X_train, y_train, mini_batch_size)
        for tokens1, tokens2, labels in g:
            embedding = nn.Embedding.from_pretrained(weights)
            tokens1 = embedding(tokens1.long())
            tokens2 = embedding(tokens2.long())
            torch.cuda.empty_cache()
            #print("!!! :", tokens1.shape)
            loss = train_data(tokens1.cuda(),tokens2.cuda(), labels.cuda(), sent_attn, 
                              sent_optimizer, loss_criterion)
            loss_full.append(loss.item())
            loss_epoch.append(loss.item())

        torch.cuda.empty_cache()      
        print ('Average training loss at this epoch..minibatch ' ,  np.mean(loss_epoch))
        vlos = check_val_loss(X_dev, y_dev, mini_batch_size, sent_attn)
        print( 'Test Loss at ',i, ' is ',vlos)
        cur2, acur,pre2, recall2 =test_accuracy_full_batch(X_test, y_test, 32,entireContext_model)
        print("f1-score(test) : ", cur2 ,  "accuracy :  ", acur, "recall :", recall2, "precision :", pre2)
        sent_attn.train()
        torch.cuda.empty_cache()
        if(cur < cur2):
            cur = cur2
            print("saving...model")
            torch.save(sent_attn.state_dict(), 'abusive_detection.pt')
            

                
            
        print("************************************************************************************************************")

        print ('Loss at %d minibatches, %d epoch,(%s) is %f' %(i, epoch_counter, timeSince(start), np.mean(loss_epoch)))
        torch.cuda.empty_cache()

        print("************************************************************************************************************")   
            
   
    return 

In [181]:
test1_f1 = []
test1_acur = []
test1_pre = []
test1_recall =[]
test1_roc=[]

test1_f1_사전 = []
test1_acur_사전 = []
test1_pre_사전 = []
test1_recall_사전 =[]
test1_roc_사전=[]

# import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, DataLoader

kfold = KFold(n_splits=9,random_state=0,shuffle=True)

print(X_train.shape, y_train.shape)

print("\nKFold**************")
aurrrrr = []
t=0
cur_a=1
c=0
for train_index, validate_index in kfold.split(X_train):
    c+=1
    if(c==9):
        print("train index:", train_index.shape, "validate index:", validate_index.shape)
        print(train_index)
        
        X_train1, X_dev = X_train[train_index], X_train[validate_index]
        y_train1, y_dev = y_train[train_index], y_train[validate_index]


        print("X_train1 index:", X_train1.shape, "X_dev index:", X_dev.shape)
        print("############################################################")
        print(X_dev)
        train_2=0
        train_1=0
        train_0=0
        test_2=0
        test_1=0
        test_0=0
        for i in y_train:
                #print(i)
            if i==1:
                train_1+=1
            elif i==0:
                train_0+=1
            elif i==2:
                train_2+=1
        for i in y_dev:
                #print(i)
            if i==1:
                test_1+=1
            elif i==0:
                test_0+=1
            elif i==2:
                test_2+=1


        print_every = 100
        vocab_size = 300000
        output_size = 1 # binary class (1 or 0)
        embedding_dim = 300
        num_filters = 100
        kernel_sizes = [3,4,5]
        num_layer=1
        num_classes =3

        entireContext_model = entireContext( )



        learning_rate = 0.001
        momentum = 0.9
        criterion = nn.CrossEntropyLoss()
        flag1="abusive_detection"+str(cur_a)+".pth"


            #final_attn_optimizer = torch.optim.SGD(final_attn.parameters(), lr=learning_rate, momentum= momentum)

        #final_attn_optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, final_attn.parameters()), lr=learning_rate)    

        entireContext_optimizer = torch.optim.Adam(list(entireContext_model.parameters()), lr=learning_rate)
        
        entireContext_model.cuda()

        print(cur_a)
        print("#######################")
        print(X_train1)
        ac,b,c,d= train_early_stopping(64, X_train, y_train, X_dev,y_dev, X_test,y_test, entireContext_model, 
                                            entireContext_optimizer, criterion,  1000)
        cur_a+=1
        print("%%%%%")
        print(ac)
        print(b)
        print(c)
        print(d)

        print("%%%")
        test1_acur.append(b)




(12329,) (12329,)

KFold**************
train index: (10960,) validate index: (1369,)
[    1     2     3 ... 12325 12327 12328]
X_train1 index: (10960,) X_dev index: (1369,)
############################################################
[list([[0, 0, 85036, 0, 392469, 1803, 27917, 472, 104, 3947, 2365, 123, 2280, 5815, 104103, 0]])
 list([[120521, 116, 1977, 501, 122245, 116, 154867, 34894, 33420, 2523, 0, 2339, 37943, 0], [1301, 39471, 3004, 479222, 754247, 13816, 10136, 7781, 0, 9198, 81732, 0, 6432, 0, 0, 416069, 506, 0]])
 list([[8489, 200558, 26, 3606, 51105, 79, 6890, 0], [0, 48645, 110159, 6981, 58, 22684, 115, 32123, 0, 49068, 0], [442, 97556, 55534, 17261, 743, 562, 16280]])
 ... list([[0, 22466, 5918, 0, 0, 0], [2393, 456581, 2751, 0]])
 list([[68639, 63049, 49257, 0, 598, 22487, 8701, 363, 78, 26106, 0], [87, 510, 151683, 0, 85331, 166413]])
 list([[7094, 59429, 4404, 76, 112948, 0, 94364, 79922, 0, 102948, 745, 0, 100777, 43036, 7150, 0, 14575, 0], [11215, 17288, 18583, 0, 366

test 갯수 : 3168
F-1score :  0.5321969696969697
AUC :  0
[1 1 1 ... 1 0 1]
############
[1 1 0 ... 1 0 0]
맞춘갯수 :  1686
f1-score(test) :  0.5321969696969697 accuracy :   0.5321969696969697 recall : 0.5321969696969697 precision : 0.5321969696969697
saving...model
************************************************************************************************************
Loss at 11 minibatches, 0 epoch,(2m 55s) is 0.470977
************************************************************************************************************
Average training loss at this epoch..minibatch  0.45941360962266725
Test Loss at  12  is  0.44229576
test 갯수 : 3168
F-1score :  0.5517676767676768
AUC :  0
[1 1 1 ... 1 1 1]
############
[0 1 0 ... 0 0 1]
맞춘갯수 :  1748
f1-score(test) :  0.5517676767676768 accuracy :   0.5517676767676768 recall : 0.5517676767676768 precision : 0.5517676767676768
saving...model
************************************************************************************************************

Average training loss at this epoch..minibatch  0.4065987481735647
Test Loss at  25  is  0.39875618
test 갯수 : 3168
F-1score :  0.6559343434343434
AUC :  0
[1 1 0 ... 1 1 1]
############
[1 1 0 ... 1 1 1]
맞춘갯수 :  2078
f1-score(test) :  0.6559343434343434 accuracy :   0.6559343434343434 recall : 0.6559343434343434 precision : 0.6559343434343434
saving...model
************************************************************************************************************
Loss at 25 minibatches, 0 epoch,(6m 38s) is 0.406599
************************************************************************************************************
Average training loss at this epoch..minibatch  0.4026460895159592
Test Loss at  26  is  0.37680435
test 갯수 : 3168
F-1score :  0.6171085858585859
AUC :  0
[1 1 1 ... 1 0 1]
############
[1 0 1 ... 1 0 0]
맞춘갯수 :  1955
f1-score(test) :  0.6171085858585859 accuracy :   0.6171085858585859 recall : 0.6171085858585859 precision : 0.6171085858585859
************************

Average training loss at this epoch..minibatch  0.3846305439559122
Test Loss at  39  is  0.3649147
test 갯수 : 3168
F-1score :  0.6174242424242424
AUC :  0
[1 0 1 ... 1 1 1]
############
[0 0 1 ... 1 0 1]
맞춘갯수 :  1956
f1-score(test) :  0.6174242424242424 accuracy :   0.6174242424242424 recall : 0.6174242424242424 precision : 0.6174242424242424
************************************************************************************************************
Loss at 39 minibatches, 0 epoch,(10m 20s) is 0.384631
************************************************************************************************************
Average training loss at this epoch..minibatch  0.3884258839922647
Test Loss at  40  is  0.36530262
test 갯수 : 3168
F-1score :  0.646780303030303
AUC :  0
[1 1 0 ... 1 1 1]
############
[1 1 0 ... 1 1 0]
맞춘갯수 :  2049
f1-score(test) :  0.646780303030303 accuracy :   0.646780303030303 recall : 0.646780303030303 precision : 0.646780303030303
********************************************

Average training loss at this epoch..minibatch  0.3712630020454526
Test Loss at  53  is  0.3604524
test 갯수 : 3168
F-1score :  0.6584595959595959
AUC :  0
[1 1 1 ... 1 1 1]
############
[0 1 1 ... 0 0 0]
맞춘갯수 :  2086
f1-score(test) :  0.6584595959595959 accuracy :   0.6584595959595959 recall : 0.6584595959595959 precision : 0.6584595959595959
************************************************************************************************************
Loss at 53 minibatches, 0 epoch,(14m 3s) is 0.371263
************************************************************************************************************
Average training loss at this epoch..minibatch  0.3699516305544724
Test Loss at  54  is  0.3607084
test 갯수 : 3168
F-1score :  0.5946969696969697
AUC :  0
[1 1 1 ... 1 1 1]
############
[1 1 0 ... 1 0 0]
맞춘갯수 :  1884
f1-score(test) :  0.5946969696969697 accuracy :   0.5946969696969697 recall : 0.5946969696969697 precision : 0.5946969696969697
*****************************************

...

Average training loss at this epoch..minibatch  0.1584384847956244
Test Loss at  207  is  0.15713142
test 갯수 : 3168
F-1score :  0.8068181818181818
AUC :  0
[1 1 0 ... 0 0 1]
############
[0 1 0 ... 0 0 0]
맞춘갯수 :  2556
f1-score(test) :  0.8068181818181818 accuracy :   0.8068181818181818 recall : 0.8068181818181818 precision : 0.8068181818181818
saving...model
************************************************************************************************************
Loss at 207 minibatches, 0 epoch,(54m 54s) is 0.158438
************************************************************************************************************
Average training loss at this epoch..minibatch  0.15985105256550014
Test Loss at  208  is  0.13844134
test 갯수 : 3168
F-1score :  0.8020833333333334
AUC :  0
[1 1 0 ... 1 1 1]
############
[0 1 0 ... 1 0 0]
맞춘갯수 :  2541
f1-score(test) :  0.8020833333333334 accuracy :   0.8020833333333334 recall : 0.8020833333333334 precision : 0.8020833333333334
*******************

Average training loss at this epoch..minibatch  0.1323787396249827
Test Loss at  221  is  0.1342432
test 갯수 : 3168
F-1score :  0.7626262626262628
AUC :  0
[1 1 1 ... 1 0 1]
############
[0 1 1 ... 0 0 0]
맞춘갯수 :  2416
f1-score(test) :  0.7626262626262628 accuracy :   0.7626262626262627 recall : 0.7626262626262627 precision : 0.7626262626262627
************************************************************************************************************
Loss at 221 minibatches, 0 epoch,(58m 37s) is 0.132379
************************************************************************************************************
Average training loss at this epoch..minibatch  0.1426039090147242
Test Loss at  222  is  0.12134808
test 갯수 : 3168
F-1score :  0.7736742424242423
AUC :  0
[1 0 0 ... 1 0 1]
############
[0 0 0 ... 1 0 1]
맞춘갯수 :  2451
f1-score(test) :  0.7736742424242423 accuracy :   0.7736742424242424 recall : 0.7736742424242424 precision : 0.7736742424242424
************************************

Average training loss at this epoch..minibatch  0.11619504327730586
Test Loss at  235  is  0.119609505
test 갯수 : 3168
F-1score :  0.7881944444444445
AUC :  0
[1 0 0 ... 1 1 1]
############
[1 0 1 ... 0 1 1]
맞춘갯수 :  2497
f1-score(test) :  0.7881944444444445 accuracy :   0.7881944444444444 recall : 0.7881944444444444 precision : 0.7881944444444444
************************************************************************************************************
Loss at 235 minibatches, 0 epoch,(62m 20s) is 0.116195
************************************************************************************************************
Average training loss at this epoch..minibatch  0.11587549117393792
Test Loss at  236  is  0.119059816
test 갯수 : 3168
F-1score :  0.7727272727272727
AUC :  0
[1 1 1 ... 1 1 0]
############
[1 1 1 ... 0 1 0]
맞춘갯수 :  2448
f1-score(test) :  0.7727272727272727 accuracy :   0.7727272727272727 recall : 0.7727272727272727 precision : 0.7727272727272727
*******************************

Average training loss at this epoch..minibatch  0.11847690859576687
Test Loss at  249  is  0.1080461
test 갯수 : 3168
F-1score :  0.7856691919191919
AUC :  0
[1 1 1 ... 1 1 1]
############
[1 0 1 ... 0 1 1]
맞춘갯수 :  2489
f1-score(test) :  0.7856691919191919 accuracy :   0.7856691919191919 recall : 0.7856691919191919 precision : 0.7856691919191919
************************************************************************************************************
Loss at 249 minibatches, 0 epoch,(66m 3s) is 0.118477
************************************************************************************************************
Average training loss at this epoch..minibatch  0.1031517924760313
Test Loss at  250  is  0.08416758
test 갯수 : 3168
F-1score :  0.7979797979797979
AUC :  0
[1 0 1 ... 1 1 0]
############
[1 0 1 ... 1 1 1]
맞춘갯수 :  2528
f1-score(test) :  0.7979797979797979 accuracy :   0.797979797979798 recall : 0.797979797979798 precision : 0.797979797979798
***************************************

KeyboardInterrupt: 

## Evaluation

In [ ]:
torch.save(weights, 'weights.pt')

In [ ]:
embed_lookup = torch.load('embed_lookup.pt')

In [292]:
def get_predictions(val_tokens1, val_tokens2,  sent_attn):
    max_sents, batch_size, max_tokens, embed_size = val_tokens1.size()
    y_pred = sent_attn(val_tokens1.cuda(),val_tokens2.cuda(),  max_sents)
    return y_pred

# convert reviews to tokens
def tokenize_all_reviews(reviews_split):
    reviews_words = reviews_split.split(' ')
    tokenized_reviews = []
    for review in reviews_words:
        ints = []
        for word in review.split(' '):
            if(word==''):
                continue
            if(word=='.'):
                continue
            try:
                idx = embed_lookup.vocab[word].index
            except: 
                idx = 0
            tokenized_reviews.append(idx)
    return tokenized_reviews



def iterate_minibatches(inputs,  batchsize, shuffle=False):
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt]

def pad_batch(mini_batch):
    mini_batch_size = len(mini_batch)
    max_sent_len = int(np.max([len(x) for x in mini_batch]))
    max_token_len = int(np.max([len(val) for sublist in mini_batch for val in sublist]))

    if(max_sent_len==1):
        max_sent_len=2
        
    if(max_sent_len >= 50):
        max_sent_len = 20
    if(max_token_len >= 300):
        max_token_len = 100
        
    main_matrix = np.zeros((mini_batch_size, max_sent_len, max_token_len), dtype= np.int)
    for i in range(main_matrix.shape[0]):
        for j in range(main_matrix.shape[1]):
            for k in range(main_matrix.shape[2]):
                try:
                    main_matrix[i,j,k] = mini_batch[i][j][k]
                except IndexError:
                    pass
    return Variable(torch.from_numpy(main_matrix).transpose(0,1))

def pad_batch2(mini_batch):
    mini_batch_size = len(mini_batch)
    max_sent_len = int(np.max([len(mini_batch[x]) for x in range(0,len(mini_batch))]))
    if(max_sent_len==1):
        max_sent_len=2
        
    if(max_sent_len >=300):
        max_sent_len = 100
        
    #print("max_sent_len:", max_sent_len)
    main_matrix = np.zeros((mini_batch_size, max_sent_len), dtype= np.int)
    for i in range(main_matrix.shape[0]):
         for k in range(main_matrix.shape[1]):
            try:
                main_matrix[i,k] = mini_batch[i][k]
            except IndexError:
                pass
    return Variable(torch.from_numpy(main_matrix))


def gen_minibatch2(tokens,  mini_batch_size, shuffle= True):
    for token in iterate_minibatches(tokens,  mini_batch_size, shuffle= shuffle):
        token1 = pad_batch(token)
        token_x=[]
        for i in token:
            c = []
            for j in i:
                for k in j:
                    c.append(k)
            token_x.append(c)

        token2 = pad_batch2(token_x)

        yield token1, token2 

In [359]:
def test_accuracy_full_batch3(a, mini_batch_size,  sent_attn):
    a_t=' '
    c_t=' '
    j=[]
    g=0
    tokens=list()
    tok=[]

    for e in a.split(' '):
        a_t +=e+' '

        c_t = list(tokenize_all_reviews(a_t))
        g+=1
        if(len(c_t) <=3):
            continue
        if('.' in e):
            b_t = list(tokenize_all_reviews(a_t))
            j.append(b_t)
            a_t=' '
        elif('?' in e):
            b_t = list(tokenize_all_reviews(a_t))
            j.append(b_t)
            a_t=' '
        elif('!' in e):
            b_t = list(tokenize_all_reviews(a_t))
            j.append(b_t)
            a_t=' '

    if(len(j) == 0 ):
        b_t = list(tokenize_all_reviews(a))
        j.append(b_t)
    elif(len(j) > 0 and len(a_t)>=3):
        b_t = list(tokenize_all_reviews(a_t))
        j.append(b_t)

    print(j)

    tokens.append(j)
    tokens.append(j)
    
    

    
    
    tokens = np.array(tokens)
    print("22tokens# :", tokens.shape)
    
    

    sent_attn.eval()

    p = []
    p2 = []
    l = []
   
    probs=[]
   
    g = gen_minibatch2(tokens, mini_batch_size)

    for token1,token2 in g:
        
        embedding = nn.Embedding.from_pretrained(weights)
        tokens1 = embedding(token1.long())
        tokens2 = embedding(token2.long())
        print("tokens1#11#: ", tokens1.shape)
        print("tokens2:", tokens2.shape)
        y_pred3 = get_predictions(tokens1,tokens2,  sent_attn)

        y_pred1=F.softmax(y_pred3)
        _, y_pred = y_pred1.max(1)
        print("##y_pred3##:", y_pred.shape)
            
        

    return y_pred   

In [ ]:

print_every = 100
vocab_size = 300000
output_size = 1 # binary class (1 or 0)
embedding_dim = 300
num_filters = 100
kernel_sizes = [3,4,5]
num_layer=1
num_classes =3

entireContext_model = entireContext( )
entireContext_model.cuda()



learning_rate = 0.001
momentum = 0.9
criterion = nn.CrossEntropyLoss()
entireContext_model.load_state_dict(torch.load("abusive_detection.pt"))
entireContext_model.eval()
cur2, acur,pre2, recall2 =test_accuracy_full_batch(X_test, y_test, 32,entireContext_model)
print("f1-score(test) : ", cur2 ,  "accuracy :  ", acur, "recall :", recall2, "precision :", pre2)



In [360]:
a="안철수 부인 다운 계약서 작성 시인 매입. 실거래가 보다 4억 원 정도.. 낮춰서 신고 약 2000만 원 세금 탈루 탈세가 드러날 경우 일벌백계로 엄중하게 처벌해서 세금을 떼먹는 것은 엄두도 내지 못하도록 해야 한다 . 안철수 본인 저서 中"
prediction =test_accuracy_full_batch3(a, 2,entireContext_model)
print(prediction[0].item())
if(prediction[0].item()==0.0):
    print("비윤리")
elif(prediction[0].item()==1.0):
    print("윤리")


[[29958, 1282, 13225, 95009, 3959, 1857, 0], [500376, 571, 0, 507, 0], [192907, 9797, 189, 0, 507, 13480, 293189, 561957, 93985, 135, 0, 130142, 0, 11215, 0, 118, 230446, 3601, 10935, 1609, 43], [29958, 5379, 3847, 1218]]
22tokens# : (2, 4)
tokens1#11#:  torch.Size([4, 2, 21, 300])
tokens2: torch.Size([2, 37, 300])
##y_pred3##: torch.Size([2])
0
비윤리


/home/dlwngud3028/venv-python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:69: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
